# Model a Snail - A simple Neural Network

### Python Future imports for compatibility with python2

In [ ]:
import sys

from __future__ import print_function, division

### Import Keras, and other necessary libraries

In [ ]:
%matplotlib inline


import tensorflow
import keras
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout
from keras import  optimizers

import numpy as np

from matplotlib import pyplot as plt
from IPython.display import clear_output

### Version Check

In [ ]:
print("Keras " , keras.__version__)
print("tensorflow, ", tensorflow.__version__)
print("python", sys.version)

# Load the Data - Snail has 2 features, to decide to eat or not

In DSX, loading data from Hadoop / DB / File is a 1-click effort

## Load the file, snail_experiment.csv

1. Upload the file snail_experiment.csv on the project overview
2. Make the next cell (below) active 
3. Add the data "Insert to code", "Insert Pandas Dataframe"
4. Rename the snail_data

In [ ]:
snail_data = df_data_4

### A hungry signal, and a food-within-range signal, will decide if the Snail will eat

In [ ]:
snail_data.head(5)

# Create Test and Train set

In [ ]:
msk = np.random.rand(len(snail_data)) < 0.8

train = snail_data[msk]
test = snail_data[~msk]

In [ ]:
# data preprocessing
Y_train = to_categorical( 1 * (train["action"] == "eat") )
Y_test = to_categorical( 1 * (test["action"] == "eat") )
X_train = train[["hunger","food"]].values
X_test = test[["hunger","food"]].values

# Jupyter Notebooks can live Plot - This code is to live see the training curve

In [ ]:
# updatable plot
# a minimal example (sort of)

class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.fig = plt.figure()
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.figure()
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

# Here we design our Neural Network, and select optimizer with Learning Rate

- Change the learning rate to see the effect.
- Add a few more layers
- Change the activation function

In [ ]:
model = Sequential()
model.add(Dense(2, activation='sigmoid', input_dim=2))

sgd = optimizers.SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# The actual training happens with the fit() 

In [ ]:
model.fit(X_train, Y_train,
          epochs=10,
          validation_data=(X_test, Y_test),
          callbacks=[plot_losses],
          verbose=0)

# We can now use our trained network to check some cases

### Snail is very hungry, but food is out of reach

In [ ]:
model.predict_classes( np.array([[1,  .1]]) , verbose=1)

### Snail is not that hungry, but food is within reach

In [ ]:
model.predict_classes( np.array([[.55,  .9]]) , verbose=1)

### Snail is not hungry, food is below him

In [ ]:
model.predict_classes( np.array([[.3,  1.0]]) , verbose=1)